In [6]:
import pandas as pd

# Load the CSV files into DataFrames
nba_17_18 = pd.read_csv("nba17-18.csv")
nba_18_19 = pd.read_csv("nba18-19.csv")
nba_19_20 = pd.read_csv("nba19-20.csv")
nba_20_21 = pd.read_csv("nba20-21.csv")
nba_21_22 = pd.read_csv("nba21-22.csv")
nba_22_23 = pd.read_csv("nba22-23.csv")
nba_23_24 = pd.read_csv("nba23-24.csv")

# Remove first and last 2 columns
def clean_data(df):
    return df.iloc[:, 1:-2]

# Clean each dataset
nba_17_18 = clean_data(nba_17_18)
nba_18_19 = clean_data(nba_18_19)
nba_19_20 = clean_data(nba_19_20)
nba_20_21 = clean_data(nba_20_21)
nba_21_22 = clean_data(nba_21_22)
nba_22_23 = clean_data(nba_22_23)
nba_23_24 = clean_data(nba_23_24)

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Joel Embiid,28,PHI,C,66,66,34.6,11.0,20.1,0.548,...,0.857,1.7,8.4,10.2,4.2,1.0,1.7,3.4,3.1,33.1
1,Luka Dončić,23,DAL,PG,66,66,36.2,10.9,22.0,0.496,...,0.742,0.8,7.8,8.6,8.0,1.4,0.5,3.6,2.5,32.4
2,Damian Lillard,32,POR,PG,58,58,36.3,9.6,20.7,0.463,...,0.914,0.8,4.0,4.8,7.3,0.9,0.3,3.3,1.9,32.2
3,Shai Gilgeous-Alexander,24,OKC,PG,68,68,35.5,10.4,20.3,0.510,...,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4
4,Giannis Antetokounmpo,28,MIL,PF,63,63,32.1,11.2,20.3,0.553,...,0.645,2.2,9.6,11.8,5.7,0.8,0.8,3.9,3.1,31.1
